In [ ]:
!pip install transformers -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from nltk.corpus import wordnet
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import pandas as pd
train_df = pd.read_json("/content/drive/MyDrive/NLP_dataset/train.jsonl", lines = True) #Replace relavant path
test_df = pd.read_json("/content/drive/MyDrive/NLP_dataset/validation.jsonl", lines = True) #Replace relavant path

In [ ]:
print("train data shape:",train_df.shape)
print("validation data shape:",test_df.shape)

train data shape: (3200, 14)
validation data shape: (800, 14)


In [ ]:
train_df_bkp = train_df.copy()
test_df_bkp = test_df.copy()

In [ ]:
train_df = train_df[["postText", "targetTitle", "targetParagraphs", "tags"]]
test_df = test_df[["postText", "targetTitle", "targetParagraphs", "tags"]]

In [ ]:
train_df.rename(columns={"postText":"Post","targetTitle":"Title", "targetParagraphs":"Content", "tags":"Spoiler_Type"}, inplace = True)
# train_df.head(2)
test_df.rename(columns={"postText":"Post","targetTitle":"Title", "targetParagraphs":"Content", "tags":"Spoiler_Type"}, inplace = True)
# test_df.head(2)

<ipython-input-9-db58f88c2b8c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.rename(columns={"postText":"Post","targetTitle":"Title", "targetParagraphs":"Content", "tags":"Spoiler_Type"}, inplace = True)


In [ ]:
def numeric_labels(spoiler_type):
  if spoiler_type[0] == "passage":
    return 1
  elif spoiler_type[0] == "phrase":
    return 0
  else:
    return 2

In [ ]:
def list_to_string_concat(TgtPara):
  res = ' '.join(TgtPara)
  return res

In [ ]:
test_df["Spoiler_Type"] = test_df["Spoiler_Type"].apply(numeric_labels)
train_df["Spoiler_Type"] = train_df["Spoiler_Type"].apply(numeric_labels)

<ipython-input-12-e209717ffdb1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["Spoiler_Type"] = train_df["Spoiler_Type"].apply(numeric_labels)


In [ ]:
train_df["Content"] = train_df["Content"].apply(list_to_string_concat)
test_df["Content"] = test_df["Content"].apply(list_to_string_concat)

<ipython-input-13-65f2695327dd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["Content"] = train_df["Content"].apply(list_to_string_concat)


In [ ]:
train_df["Post"] = train_df["Post"].apply(list_to_string_concat)
test_df["Post"] = test_df["Post"].apply(list_to_string_concat)

<ipython-input-14-e31bc68e4cac>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["Post"] = train_df["Post"].apply(list_to_string_concat)


In [ ]:
train_df = train_df.fillna('')
test_df = test_df.fillna('')

In [ ]:
#Lower-case:
train_df['Post'] = train_df['Post'].str.lower()
train_df['Title'] = train_df['Title'].str.lower()
train_df['Content'] = train_df['Content'].str.lower()
#Lower-case:
test_df['Post'] = test_df['Post'].str.lower()
test_df['Title'] = test_df['Title'].str.lower()
test_df['Content'] = test_df['Content'].str.lower()

In [ ]:
df_multi = train_df.copy()

In [ ]:
df_multi = df_multi[df_multi['Spoiler_Type']==2]

In [ ]:
df_multi.reset_index(drop=True,inplace=True)

In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm')

def get_synonym(word):
    """
    Get the most suitable synonym for a word
    """
    synonyms = []
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.append(lemma.name())
    
    # Calculate the similarity scores of all synonyms to the original word
    word_similarities = []
    for synonym in synonyms:
        word_similarities.append((synonym, nlp(word).similarity(nlp(synonym))))
    
    # Sort synonyms by their similarity score
    word_similarities.sort(key=lambda x: x[1], reverse=True)
    
    # Select the first synonym if there is at least one with a non-zero similarity score
    for syn in word_similarities:
        if word != syn[0] and syn[1] > 0:
            syn_dict[word] = syn[0]
            return syn[0]
    
    # Return the original word if there are no suitable synonyms
    syn_dict[word] = word
    return word

In [ ]:
from collections import defaultdict
syn_dict = defaultdict(str)
def change_synonyms(sentence):
  doc = nlp(sentence)
  new_sentence = []
  for token in doc:
      if token.pos_ == "NOUN":
          if token.text in syn_dict.keys():
              new_sentence.append(syn_dict[token.text])
          else:
              new_sentence.append(get_synonym(token.text))
      else:
          new_sentence.append(token.text)

  altered_sentence = ' '.join(new_sentence)
  return altered_sentence

In [ ]:
# import pandas as pd
# df_multi = pd.read_csv('/content/drive/MyDrive/NLP_dataset/multi_part_train_data.csv')

In [ ]:
df_multi['Post'] = df_multi['Post'].apply(change_synonyms)

In [ ]:
df_multi['Title'] = df_multi['Title'].apply(change_synonyms)

In [ ]:
df_multi['Content'] = df_multi['Content'].apply(change_synonyms)

In [ ]:
# save the dataframe to a CSV file with headers
df_multi.to_csv('multi_part_train_data_new.csv', index=False, header=True)

In [ ]:
#Lower-case:
df_multi['Post'] = df_multi['Post'].str.lower()
df_multi['Title'] = df_multi['Title'].str.lower()
df_multi['Content'] = df_multi['Content'].str.lower()

In [ ]:
df_merged = pd.concat([train_df, df_multi], ignore_index=True, sort=False)

In [ ]:
train_df_bkp = train_df.copy()

In [ ]:
train_df = df_merged.copy()

In [ ]:
len(train_df)

3759

In [ ]:
train_df["concat_input"] = train_df[['Post','Content','Title']].apply(lambda x: x['Post'] + ' ? ' + x['Title'] + ' -- ' + x['Content'], axis=1)
test_df["concat_input"] = test_df[['Post','Content','Title']].apply(lambda x: x['Post'] + ' ? ' + x['Title'] + ' -- ' + x['Content'], axis=1)

In [ ]:
train_df.drop(["Post"], inplace = True, axis = 1)
train_df.drop(["Content"], inplace = True, axis = 1)
train_df.drop(["Title"], inplace = True, axis = 1)
test_df.drop(["Post"], inplace = True, axis = 1)
test_df.drop(["Content"], inplace = True, axis = 1)
test_df.drop(["Title"], inplace = True, axis = 1)

In [ ]:
train_df['Spoiler_Type'].value_counts()

0    1367
1    1274
2    1118
Name: Spoiler_Type, dtype: int64

In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

import gc
gc.collect()
torch.cuda.empty_cache() 

cuda


In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# Load pre-trained RoBERTa model and tokenizer
model_name = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=3).to(device)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

In [ ]:
from sklearn.model_selection import train_test_split
X_train = list(train_df["concat_input"])
y_train = list(train_df["Spoiler_Type"])
X_test = list(test_df["concat_input"])
y_test = list(test_df["Spoiler_Type"])
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1,stratify=y_train)
# X_test, X_val, y_test, y_val = train_test_split(X_val, y_val, test_size = 0.5, stratify=y_val)

In [ ]:
print(len(X_train))
print(len(X_test))
# print(len(X_val))

3759
800


In [ ]:
import numpy as np

# Convert lists to numpy arrays
X_train = np.array(X_train)
# X_val = np.array(X_val)
y_train = np.array(y_train)
# y_val = np.array(y_val)

# Prepare training and validation data
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=512)
# val_encodings = tokenizer(X_val.tolist(), truncation=True, padding=True, max_length=512)
train_labels = torch.tensor(y_train)
# val_labels = torch.tensor(y_val)

In [ ]:
train_input_ids = torch.tensor(train_encodings['input_ids'])
train_attention_mask = torch.tensor(train_encodings['attention_mask'])
train_labels = torch.tensor(y_train).to(device)

train_dataset = torch.utils.data.TensorDataset(train_input_ids.to(device), train_attention_mask.to(device), train_labels)

In [ ]:
# val_input_ids = torch.tensor(val_encodings['input_ids'])
# val_attention_mask = torch.tensor(val_encodings['attention_mask'])
# val_labels = torch.tensor(y_val).to(device)

# val_dataset = torch.utils.data.TensorDataset(val_input_ids.to(device), val_attention_mask.to(device), val_labels)

In [ ]:
# Set up optimizer and learning rate scheduler
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
optimizer = torch.optim.Adam(model.parameters(), lr=4e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

# Set up training loop
epochs = 5
for epoch in range(epochs):
    # Train model for one epoch
    model.train()
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
    for batch in train_loader:
        optimizer.zero_grad()
        inputs = {'input_ids': batch[0].to(device), 'attention_mask': batch[1].to(device), 'labels': batch[2].to(device)}
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # # Evaluate model on validation set
    # model.eval()
    # val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=False)
    # with torch.no_grad():
    #     num_correct = 0
    #     num_total = 0
    #     for batch in val_loader:
    #         inputs = {'input_ids': batch[0].to(device), 'attention_mask': batch[1].to(device), 'labels': batch[2].to(device)}
    #         outputs = model(**inputs)
    #         logits = outputs.logits
    #         preds = torch.argmax(logits, dim=1)
    #         num_correct += torch.sum(preds == batch[2])
    #         num_total += len(batch[2])
    #     acc = num_correct / num_total
    #     print(f'Epoch {epoch+1} - val accuracy: {acc:.4f}')

    # Update learning rate scheduler
    scheduler.step()

In [ ]:
X_test = np.array(X_test)
y_test = np.array(y_test)

# Prepare training and validation data
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True, max_length=512)
test_labels = torch.tensor(y_test)

test_input_ids = torch.tensor(test_encodings['input_ids'])
test_attention_mask = torch.tensor(test_encodings['attention_mask'])
test_labels = torch.tensor(y_test).to(device)

test_dataset = torch.utils.data.TensorDataset(test_input_ids.to(device), test_attention_mask.to(device), test_labels)

In [ ]:
# Iterate over the evaluation dataset and make predictions
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

model.eval()
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)
y_true = []
y_pred = []
with torch.no_grad():
    num_correct = 0
    num_total = 0
    for batch in test_loader:
        inputs = {'input_ids': batch[0].to(device), 'attention_mask': batch[1].to(device), 'labels': batch[2].to(device)}
        outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)
        y_true.extend(batch[2].tolist())
        y_pred.extend(preds.tolist())
        num_correct += torch.sum(preds == batch[2])
        num_total += len(batch[2])

# Calculate the accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy:.4f}')
# Calculate the F1 score
f1 = f1_score(y_true, y_pred, average='weighted')
print(f'F1 score: {f1:.4f}')
#Prev acc
acc = num_correct / num_total
print(f'Test accuracy: {acc:.4f}')

Accuracy: 0.6963
F1 score: 0.6959
Test accuracy: 0.6962


In [ ]:
# # Evaluate model on test set
# model.eval()
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)
# with torch.no_grad():
#     num_correct = 0
#     num_total = 0
#     for batch in test_loader:
#         inputs = {'input_ids': batch[0].to(device), 'attention_mask': batch[1].to(device), 'labels': batch[2].to(device)}
#         outputs = model(**inputs)
#         logits = outputs.logits
#         preds = torch.argmax(logits, dim=1)
#         num_correct += torch.sum(preds == batch[2])
#         num_total += len(batch[2])
#     acc = num_correct / num_total
#     print(f'Test accuracy: {acc:.4f}')

In [ ]:
# model_path = '/content/drive/MyDrive/NLP_dataset/roberta_classification_model.pt'
# torch.save(model.state_dict(), model_path)